In [1]:
import pdfplumber
import pandas as pd

filename = "DTU_vaegt_tbl1.pdf"

### Simple approach

Notes: 37 rows in original table
* 34 single rows
* 3 double rows
* i.e., 40 in total

In [ ]:
def pprint(matrix):
    for row in matrix:
        for element in row:
            # print(element, end="\t")  # Use "\t" for tab spacing
            print(f'"{element}"', end="\t")  # Use "\t" for tab spacing
        print()  # Move to the next line after each row


with pdfplumber.open(filename) as pdf:
    print(pdf.pages)
    page = pdf.pages[0]

    selected_rects = []

    #Select rects
    for i, rect in enumerate(page.rects):
        if rect['height'] < 1 or rect['width'] < 1:
            selected_rects.append(rect)
        # if i == 20: break #FOr testing

    page_tables = page.extract_tables({
        "vertical_strategy": "explicit",
        "horizontal_strategy": "explicit",
        "explicit_vertical_lines": selected_rects,
        "explicit_horizontal_lines": selected_rects,
    })

    table = page_tables[0]
    placeholder_cols = ["1","2","3","4","5"]
    df = pd.DataFrame(table, columns= placeholder_cols)
    print(df.to_markdown())

[<Page:1>]
|    | 1                                                | 2                           | 3         | 4         | 5   |
|---:|:-------------------------------------------------|:----------------------------|:----------|:----------|:----|
|  0 | Brød                                             | Vægt pr stk. eller skive, g |           |           |     |
|    |                                                  | Lille Mellem Stor Kilde     |           |           |     |
|  1 | Bagel                                            |                             | 90        |           | d   |
|  2 | Bolle, almindelig/fuldkorn - bager/industribolle | 40                          | 60        | 80        | d   |
|  3 | Bolle, almindelig/fuldkorn - hjemmebagt          | 50                          | 70        | 120       | r   |
|  4 | Bondebrød, skive                                 |                             | 50        |           | d   |
|  5 | Burger/hamburger bolle, almindelig/ful

In [ ]:
#Drawing 
cells = []

with pdfplumber.open(filename) as pdf:
    print(pdf.pages)
    page = pdf.pages[0]
    element_type = page.edges
    #TODO: 
    im = page.to_image(resolution = 400)
    for i in range(0,len(element_type)):
    # for i in range(0,20):
    #Selected rects
        rect = element_type[i]
        # if rect['height'] < 1 or rect['width'] < 1: continue #1 Ignore small rectangles
        cells.append(rect)
    im = im.draw_rects(cells)
    im.show()

[<Page:1>]
